In [1]:
from datetime import datetime
import os
import hopsworks

if os.environ.get("HOPSWORKS_API_KEY") is None:
    os.environ["HOPSWORKS_API_KEY"] = open(".hw_key").read()

project = hopsworks.login()
fs = project.get_feature_store()

2024-12-16 13:58:16,972 INFO: Initializing external client
2024-12-16 13:58:16,972 INFO: Base URL: https://c.app.hopsworks.ai:443
2024-12-16 13:58:18,447 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1157271


## Feature View Prep

In [9]:
delays_fg = fs.get_feature_group(
    name='delays',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

In [15]:
# Join delays and weather feature groups on arrival_time_bin and date respectively
selected_features = delays_fg.select_all().join(
    weather_fg.select_all(),
    left_on=['arrival_time_bin'],
    right_on=['date'],
    join_type='inner'
)
selected_features.show(10)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.52s) 


,arrival_time_bin,mean_arrival_delay_seconds,max_arrival_delay_seconds,mean_departure_delay_seconds,max_departure_delay_seconds,on_time_mean_percent,stop_count,date,temperature_2m,apparent_temperature,precipitation,rain,snowfall,snow_depth,cloud_cover,wind_speed_10m,wind_speed_100m,wind_gusts_10m
0,2024-09-01 14:00:00+00:00,105.430142,1178.0,118.684861,1313.0,86.251931,1984,2024-09-01 14:00:00+00:00,13.860500,13.208918,0.9,0.9,0.0,0.0,100.0,5.191994,8.161764,19.799999
1,2024-10-01 08:00:00+00:00,103.818072,3046.0,116.391907,3046.0,87.704609,4337,2024-10-01 08:00:00+00:00,9.110500,7.225355,0.0,0.0,0.0,0.0,8.0,5.257109,8.942214,14.400000
2,2024-10-01 18:00:00+00:00,77.122595,3594.0,87.217626,3610.0,89.995418,3053,2024-10-01 18:00:00+00:00,7.460500,4.970199,0.0,0.0,0.0,0.0,97.0,9.257321,19.669479,15.119999
3,2024-10-01 22:00:00+00:00,446.519496,661.0,449.106932,671.0,7.799784,60,2024-10-01 22:00:00+00:00,4.310500,1.708803,0.0,0.0,0.0,0.0,97.0,7.562142,10.304114,9.000000
4,2024-09-01 04:00:00+00:00,44.264062,897.0,55.250783,897.0,90.115961,747,2024-09-01 04:00:00+00:00,11.610500,8.882175,0.0,0.0,0.0,0.0,98.0,14.291592,26.785547,29.519999
5,2024-09-01 16:00:00+00:00,83.868337,1189.0,97.162893,1455.0,87.901219,1995,2024-09-01 16:00:00+00:00,12.710501,13.101048,0.1,0.1,0.0,0.0,100.0,1.609969,3.075841,10.440001
6,2024-10-01 06:00:00+00:00,115.058707,1486.0,127.418248,1626.0,86.581264,6050,2024-10-01 06:00:00+00:00,5.860500,3.418180,0.0,0.0,0.0,0.0,77.0,7.491114,16.179987,14.400000
7,2024-10-01 17:00:00+00:00,73.469295,3594.0,83.868246,3610.0,89.691676,3181,2024-10-01 17:00:00+00:00,8.260500,5.731498,0.0,0.0,0.0,0.0,100.0,9.821140,18.965031,15.119999
8,2024-09-01 02:00:00+00:00,0.000000,0.0,0.000000,0.0,0.000000,0,2024-09-01 02:00:00+00:00,11.910501,9.150716,0.0,0.0,0.0,0.0,82.0,14.255272,26.649727,28.799999
9,2024-09-01 10:00:00+00:00,57.773472,2178.0,72.342519,2178.0,91.649572,1998,2024-09-01 10:00:00+00:00,17.060501,14.613287,0.2,0.2,0.0,0.0,100.0,11.609651,16.394829,33.480000


In [2]:
feature_view = fs.get_or_create_feature_view(
    name='delays_fv',
    description="weather features with delays as the target",
    version=1,
    labels=['mean_arrival_delay_seconds', 'max_arrival_delay_seconds', 'mean_departure_delay_seconds', 'max_departure_delay_seconds', 'on_time_mean_percent'],
    query=selected_features,
)

NameError: name 'selected_features' is not defined

## Data Split

In [5]:
feature_view = fs.get_feature_view(name='delays_fv', version=1)

start_date_test_data = "2024-10-01"
# Convert string to datetime object
test_start = datetime.strptime(start_date_test_data, "%Y-%m-%d")
X_train, X_test, y_train, y_test = feature_view.train_test_split(
    test_start=test_start
)
X_train

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.76s) 


,arrival_time_bin,stop_count,date,temperature_2m,apparent_temperature,precipitation,rain,snowfall,snow_depth,cloud_cover,wind_speed_10m,wind_speed_100m,wind_gusts_10m
0,2024-09-01 14:00:00+00:00,1984,2024-09-01 14:00:00+00:00,13.860500,13.208918,0.9,0.9,0.0,0.0,100.0,5.191994,8.161764,19.799999
4,2024-09-01 04:00:00+00:00,747,2024-09-01 04:00:00+00:00,11.610500,8.882175,0.0,0.0,0.0,0.0,98.0,14.291592,26.785547,29.519999
5,2024-09-01 16:00:00+00:00,1995,2024-09-01 16:00:00+00:00,12.710501,13.101048,0.1,0.1,0.0,0.0,100.0,1.609969,3.075841,10.440001
8,2024-09-01 02:00:00+00:00,0,2024-09-01 02:00:00+00:00,11.910501,9.150716,0.0,0.0,0.0,0.0,82.0,14.255272,26.649727,28.799999
9,2024-09-01 10:00:00+00:00,1998,2024-09-01 10:00:00+00:00,17.060501,14.613287,0.2,0.2,0.0,0.0,100.0,11.609651,16.394829,33.480000
10,2024-09-01 22:00:00+00:00,1,2024-09-01 22:00:00+00:00,11.710501,10.263132,0.9,0.9,0.0,0.0,97.0,13.450708,25.407431,29.160000
12,2024-09-01 15:00:00+00:00,1827,2024-09-01 15:00:00+00:00,13.110500,12.857162,0.7,0.7,0.0,0.0,100.0,5.154415,9.028754,12.599999
15,2024-09-01 19:00:00+00:00,1466,2024-09-01 19:00:00+00:00,12.660501,11.548429,0.0,0.0,0.0,0.0,99.0,11.043261,20.545246,20.880001
16,2024-09-30 23:00:00+00:00,0,2024-09-30 23:00:00+00:00,6.160500,3.170447,0.0,0.0,0.0,0.0,99.0,10.765445,22.956175,20.160000
19,2024-09-01 08:00:00+00:00,2141,2024-09-01 08:00:00+00:00,15.610500,12.526066,0.0,0.0,0.0,0.0,38.0,16.394829,23.732710,37.799999


In [6]:
train_features = X_train.drop(['date', 'arrival_time_bin'], axis=1)
test_features = X_test.drop(['date', 'arrival_time_bin'], axis=1)

In [ ]:
y_train

## Modeling

In [11]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score

# List of labels to predict
labels = ['mean_arrival_delay_seconds', 'max_arrival_delay_seconds', 'mean_departure_delay_seconds', 'max_departure_delay_seconds', 'on_time_mean_percent']

# Train a single model for all labels
xgb_regressor = XGBRegressor()
xgb_regressor.fit(train_features, y_train[labels])

y_pred = xgb_regressor.predict(test_features)

mse_scores = {}
r2_scores = {}
for i, label in enumerate(labels):
    mse = mean_squared_error(y_test[label], y_pred[:, i])
    mse_scores[label] = mse
    print(f"MSE for {label}: {mse}")

    r2 = r2_score(y_test[label], y_pred[:, i])
    r2_scores[label] = r2
    print(f"R squared for {label}: {r2}")

MSE for mean_arrival_delay_seconds: 205846.0963253775
R squared for mean_arrival_delay_seconds: -29.674403373850872
MSE for max_arrival_delay_seconds: 3514790.5878668237
R squared for max_arrival_delay_seconds: -1.4689011205215912
MSE for mean_departure_delay_seconds: 486542.0487162522
R squared for mean_departure_delay_seconds: -72.13713775757105
MSE for max_departure_delay_seconds: 2002854.8522906324
R squared for max_departure_delay_seconds: -0.3974103043396182
MSE for on_time_mean_percent: 65.56538204564511
R squared for on_time_mean_percent: 0.9389948743236946


In [12]:
# Dictionary to store models and their predictions
models = {}
predictions = {}
mse_scores = {}
r2_scores = {}

# Train a separate model for each label
for label in labels:
    xgb_regressor = XGBRegressor()
    xgb_regressor.fit(train_features, y_train[label])
    models[label] = xgb_regressor

    # Predicting target values on the test set
    y_pred = xgb_regressor.predict(test_features)
    predictions[label] = y_pred

    # Calculating Mean Squared Error (MSE) using sklearn
    mse = mean_squared_error(y_test[label], y_pred)
    mse_scores[label] = mse
    print(f"MSE for {label}: {mse}")

    # Calculating R squared using sklearn
    r2 = r2_score(y_test[label], y_pred)
    r2_scores[label] = r2
    print(f"R squared for {label}: {r2}")

MSE for mean_arrival_delay_seconds: 206983.3674779838
R squared for mean_arrival_delay_seconds: -29.84387520112005
MSE for max_arrival_delay_seconds: 2694128.4310586923
R squared for max_arrival_delay_seconds: -0.8924418214932102
MSE for mean_departure_delay_seconds: 487558.1068873871
R squared for mean_departure_delay_seconds: -72.28987190794525
MSE for max_departure_delay_seconds: 1858340.0530044232
R squared for max_departure_delay_seconds: -0.29658099590463216
MSE for on_time_mean_percent: 74.59955317346824
R squared for on_time_mean_percent: 0.9305890551575619
